In [ ]:
import subprocess
import json

#Configure Clingo parameters

clingo_path = 'C:\\Users\\Alex\\Documents\\MSc Computer Science\\ASP\\clingo.exe'
clingo_options = ['--outf=2','-n 0']
clingo_command = [clingo_path] + clingo_options

#Solve the ruleset.
def solve(program):
    input = program.encode()
    process = subprocess.Popen(clingo_command, stdin=subprocess.PIPE, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    output, error = process.communicate(input)
    result = json.loads(output.decode())
    if result['Result'] == 'SATISFIABLE':
        return [value['Value'] for value in result['Call'][0]['Witnesses']]
    else:
        return None

In [ ]:
#Solve the problem in Clingo, specifying what kind of pattern will be generated.
#Print the number of solutions.
def generate_solutions(pattern_type):
    if pattern_type == "conventional":
        problem = open('rules\\conventional.lp', 'r').read()
    elif pattern_type == "experimental":
        problem = open('rules\\experimental.lp', 'r').read()
    solutions = solve(problem)
    #Print the number of patterns found.
    print(str(len(solutions)) + " patterns have been found.\n")

    return solutions

In [ ]:
#Represent hit_list as a table.

import pandas as pd
import matplotlib

def print_hits(hit_list):
    kick_list = [' '] * 16
    snare_list = [' '] * 16
    hat_list = [' '] * 16
    perc_list = [' '] * 16

    for i in hit_list:
        if i[0] == 'k':
            kick_list[int(i[1])-1] = 'x'
        if i[0] == 's':
            snare_list[int(i[1])-1] = 'x'
        if i[0] == 'h':
            hat_list[int(i[1])-1] = 'x'
        if i[0] == 'p':
            perc_list[int(i[1])-1] = 'x'

    data = [perc_list, hat_list, snare_list, kick_list]
    df = pd.DataFrame(data, index = ['Percussion', 'Hat', 'Snare', 'Kick'], columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16'])
    print(df)

In [ ]:
#Create MIDI representation of the drum pattern.

from midiutil import MIDIFile

#offset needed to convert to MIDI time.
offset = 0.25
track    = 0
channel  = 0
duration = 0.25 # In beats
tempo    = 174  # In BPM
volume   = 100  # 0-127, as per the MIDI standard

#Write a MIDI file for a set of hits with a given file name.
def write_midi(hit_list, file_name):
    MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
                          # automatically)
    MyMIDI.addTempo(track, 0, tempo)

    #Add kicks to middle C, snares to C#3 and hats to D3.
    for i in hit_list:
        if i[0] == 'k':
            MyMIDI.addNote(track, channel, 60, float(i[1])/4-offset, duration, volume)
        if i[0] == 's':
            MyMIDI.addNote(track, channel, 61, float(i[1])/4-offset, duration, volume)
        if i[0] == 'h':
            MyMIDI.addNote(track, channel, 62, float(i[1])/4-offset, duration, volume)
        if i[0] == 'p':
            MyMIDI.addNote(track, channel, 63, float(i[1])/4-offset, duration, volume)
            
    with open(file_name, "wb") as output_file:
        MyMIDI.writeFile(output_file)

In [ ]:
#Generate n random patterns from the answer set, depending on the pattern_type, print and store them as MIDI files.
from random import randint

def generate_patterns(pattern_type, n):
    solutions = generate_solutions(pattern_type)
    for i in range(1, n+1):
        rand_solution = solutions[randint(0, len(solutions))][:]
        #Converting the random solution into a (hit, quarter-beat) list of type (char, char).
        hit_list = []
        for hit in rand_solution:
            hit_list.append(hit[hit.find("(")+1:hit.find(")")].split(","))
        
        print_hits(hit_list)
        print("\n")
        write_midi(hit_list, "rand_pattern_" + str(i) +".mid")

In [ ]:
generate_patterns("conventional", 10)